<a href="https://colab.research.google.com/github/sadbacon1148/DataCollectionFromMySQLnRESTAPI/blob/master/DATA_COLLECTION_FROM_DB_AND_REST_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data from MySQL database

## Install PyMySQL 
which is the package to connect with MySQL database

In [ ]:
! pip install pymysql # ! means run the bash command

     |████████████████████████████████| 51kB 1.9MB/s 


In [ ]:
! ls  # list all in bash

drive  output.csv  sample_data


In [ ]:
! echo "username" > env.txt
! echo "not-an-actual-password-ofc" >> env.txt

In [ ]:
with open("env.txt",'r') as file:
  secret = file.readlines()

secret  # you gotta strip the \n out

## Config DB credential



In [ ]:
class Config:  # normally we'll keep in config file
  MYSQL_HOST = '' #confidential
  MYSQL_PORT = 3306  # default port ของ MySQL คือ 3306
  # MYSQL_USER = '' #confidential
  MYSQL_USER = secret[0].strip() 
  MYSQL_PASSWORD = secret[1].strip()
  MYSQL_DB = 'detraining'
  MYSQL_CHARSET = 'utf8mb4'
  
print(Config.MYSQL_USER)

## Connect to DB
after we obtain the credentials, we then can load data from the table

In [ ]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

In [ ]:
connection # is an object called connection

ตัวแปร connection นี้ เราได้ connect ต่อเข้ากับ database เอาไว้แล้ว
## List Tables
เรามาลองดูกันว่ามี table อะไรในนั้นบ้าง

In [ ]:
# list all tables
cursor = connection.cursor() #เปนหลักการการใช้database ทั่วๆไป ไม่ว่าจะเปน lib ไหนๆ คือต้องสร้าง cursor มาก่อนทุกครั้ง แล้วเอา cursor นี้ไป query
                             # query เสดมา fetch result จาก cursor ที่สร้างไว้ในตัวแปรนึง แล้วเราก็ปิด cursor นั้น
cursor.execute("show tables;") # show tables is in SQL language
tables = cursor.fetchall()
cursor.close()
print(tables) 

[{'Tables_in_detraining': 'current_dept_emp'}, {'Tables_in_detraining': 'departments'}, {'Tables_in_detraining': 'dept_emp'}, {'Tables_in_detraining': 'dept_emp_latest_date'}, {'Tables_in_detraining': 'dept_manager'}, {'Tables_in_detraining': 'employees'}, {'Tables_in_detraining': 'online_retail'}, {'Tables_in_detraining': 'salaries'}, {'Tables_in_detraining': 'titles'}]


In [ ]:
# ใข้ with statement แทน cursor.close()
# TODO: มาลอง Query ข้อมูลทั้งหมดจาก table online_retail ดูกัน

with connection.cursor() as cursor:
  # Read a single record
  sql = "SELECT * FROM online_retail;"
  cursor.execute(sql)
  result = cursor.fetchall()

print("number of rows: ", len(result), "⁀⊙﹏☉⁀") #len(result) = count row pa

number of rows:  161331 ⁀⊙﹏☉⁀


In [ ]:
type(result) # list ที่มี 10k rows ก็ถือว่าเยอะอยู่นะะ

list

## Convert to Pandas

In [ ]:
import pandas as pd

In [ ]:
retail = pd.DataFrame(result)

In [ ]:
type(retail)

pandas.core.frame.DataFrame

In [ ]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,19/4/2019 15:44,8.29,NaN,United Kingdom
161327,550638,22961,JAM MAKING SET PRINTED,1,19/4/2019 15:44,1.45,NaN,United Kingdom
161328,550638,22978,PANTRY ROLLING PIN,1,19/4/2019 15:44,7.46,NaN,United Kingdom
161329,550638,22982,PANTRY PASTRY BRUSH,1,19/4/2019 15:44,2.46,NaN,United Kingdom




---


# Get data from REST API

Package `requests` is needed in order to use REST API

(normally we would need to install this package but colab has already pre-install for us)



In [ ]:
import requests

ลองคลิกดูผลลัพธ์ผ่าน web browser ได้ [Currency conversion API](https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all) 

ผลลัพธ์ที่ return กลับมาจะเป็นประเภท JSON
จึงต้องใช้ package `json` (built-in) เพื่อโหลดข้อมูลเป็น dictionary

การที่เราสามารถยิง request และ output ออกมาได้เลยโดยที่ไม่ต้องสร้าง payload เพิ่ม ดังตัวอย่างนี้ เรียกว่า HTTP GET (ในกรณีอื่น ๆ สามารถเพิ่ม arguement หรือ query string เข้าไปใน URL ได้)

## Requests library
สามารถศึกษาวิธีการสร้าง request และการใช้งาน package `requests` [ได้ที่นี่](https://requests.readthedocs.io/en/master/)

In [ ]:
url = "https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all"

response = requests.get(url)
response.text

'{"Rate":{"2018-01-01T00:00:00.000Z":43.9991,"2018-01-02T00:00:00.000Z":44.0731,"2018-01-03T00:00:00.000Z":43.7025,"2018-01-04T00:00:00.000Z":43.6914,"2018-01-05T00:00:00.000Z":43.649,"2018-01-06T00:00:00.000Z":43.649,"2018-01-07T00:00:00.000Z":43.6448,"2018-01-08T00:00:00.000Z":43.7282,"2018-01-09T00:00:00.000Z":43.688,"2018-01-10T00:00:00.000Z":43.4071,"2018-01-11T00:00:00.000Z":43.3446,"2018-01-12T00:00:00.000Z":43.7705,"2018-01-13T00:00:00.000Z":43.6158,"2018-01-14T00:00:00.000Z":43.8395,"2018-01-15T00:00:00.000Z":44.0286,"2018-01-16T00:00:00.000Z":44.04,"2018-01-17T00:00:00.000Z":44.2412,"2018-01-18T00:00:00.000Z":44.322,"2018-01-19T00:00:00.000Z":44.1344,"2018-01-20T00:00:00.000Z":44.1341,"2018-01-21T00:00:00.000Z":44.2301,"2018-01-22T00:00:00.000Z":44.5456,"2018-01-23T00:00:00.000Z":44.5391,"2018-01-24T00:00:00.000Z":44.8738,"2018-01-25T00:00:00.000Z":44.5501,"2018-01-26T00:00:00.000Z":44.3559,"2018-01-27T00:00:00.000Z":44.3559,"2018-01-28T00:00:00.000Z":44.351,"2018-01-29T00:00

In [ ]:
response.json()

{'Rate': {'2018-01-01T00:00:00.000Z': 43.9991,
  '2018-01-02T00:00:00.000Z': 44.0731,
  '2018-01-03T00:00:00.000Z': 43.7025,
  '2018-01-04T00:00:00.000Z': 43.6914,
  '2018-01-05T00:00:00.000Z': 43.649,
  '2018-01-06T00:00:00.000Z': 43.649,
  '2018-01-07T00:00:00.000Z': 43.6448,
  '2018-01-08T00:00:00.000Z': 43.7282,
  '2018-01-09T00:00:00.000Z': 43.688,
  '2018-01-10T00:00:00.000Z': 43.4071,
  '2018-01-11T00:00:00.000Z': 43.3446,
  '2018-01-12T00:00:00.000Z': 43.7705,
  '2018-01-13T00:00:00.000Z': 43.6158,
  '2018-01-14T00:00:00.000Z': 43.8395,
  '2018-01-15T00:00:00.000Z': 44.0286,
  '2018-01-16T00:00:00.000Z': 44.04,
  '2018-01-17T00:00:00.000Z': 44.2412,
  '2018-01-18T00:00:00.000Z': 44.322,
  '2018-01-19T00:00:00.000Z': 44.1344,
  '2018-01-20T00:00:00.000Z': 44.1341,
  '2018-01-21T00:00:00.000Z': 44.2301,
  '2018-01-22T00:00:00.000Z': 44.5456,
  '2018-01-23T00:00:00.000Z': 44.5391,
  '2018-01-24T00:00:00.000Z': 44.8738,
  '2018-01-25T00:00:00.000Z': 44.5501,
  '2018-01-26T00:00:00.

In [ ]:
type(response.text)

str

## JSON loading
we gotta convert string in json file into dictionary datatype

In [ ]:
import json

In [ ]:
result = response.json()
result

{'Rate': {'2018-01-01T00:00:00.000Z': 43.9991,
  '2018-01-02T00:00:00.000Z': 44.0731,
  '2018-01-03T00:00:00.000Z': 43.7025,
  '2018-01-04T00:00:00.000Z': 43.6914,
  '2018-01-05T00:00:00.000Z': 43.649,
  '2018-01-06T00:00:00.000Z': 43.649,
  '2018-01-07T00:00:00.000Z': 43.6448,
  '2018-01-08T00:00:00.000Z': 43.7282,
  '2018-01-09T00:00:00.000Z': 43.688,
  '2018-01-10T00:00:00.000Z': 43.4071,
  '2018-01-11T00:00:00.000Z': 43.3446,
  '2018-01-12T00:00:00.000Z': 43.7705,
  '2018-01-13T00:00:00.000Z': 43.6158,
  '2018-01-14T00:00:00.000Z': 43.8395,
  '2018-01-15T00:00:00.000Z': 44.0286,
  '2018-01-16T00:00:00.000Z': 44.04,
  '2018-01-17T00:00:00.000Z': 44.2412,
  '2018-01-18T00:00:00.000Z': 44.322,
  '2018-01-19T00:00:00.000Z': 44.1344,
  '2018-01-20T00:00:00.000Z': 44.1341,
  '2018-01-21T00:00:00.000Z': 44.2301,
  '2018-01-22T00:00:00.000Z': 44.5456,
  '2018-01-23T00:00:00.000Z': 44.5391,
  '2018-01-24T00:00:00.000Z': 44.8738,
  '2018-01-25T00:00:00.000Z': 44.5501,
  '2018-01-26T00:00:00.

In [ ]:
#use Python Requests to convert into json instead
result_conversion_rate = json.loads(response.text)
result_conversion_rate

Check the datatype in this step cos if it isnt dict then the exception error will appear here


In [ ]:
print(type(result_conversion_rate))

# check if this result_conversion_rate is actually in dict datatype
assert isinstance(result_conversion_rate, dict)

<class 'dict'>


 ## Convert to Pandas


In [ ]:
# convert into Pandas again

conversion_rate = pd.DataFrame.from_dict(result_conversion_rate)

In [ ]:
conversion_rate[:3] #0~2

,Rate
2018-01-01T00:00:00.000Z,43.9991
2018-01-02T00:00:00.000Z,44.0731
2018-01-03T00:00:00.000Z,43.7025


rename index into date

In [ ]:
conversion_rate = conversion_rate.reset_index().rename(columns={"index":"date"})
conversion_rate[:3]

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025


In [ ]:
conversion_rate[:3]

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025


# Join the data

ในตอนนี้เราจะนำข้อมูลการซื้อขายและข้อมูล Rate การแปลงค่าเงิน เราจะรวมข้อมูลจากทั้งสอง Dataframe มารวมกัน

เราจะนำข้อมูลจากทั้งสองมารวมกันผ่าน column InvoiceDate ใน retail และ date ใน conversion_rate 

แต่ถ้าสังเกตดี ๆ แล้วจะพบว่า InvoiceDate ใน retail จะเก็บข้อมูลในรูปแบบ datetime ส่วน date ใน conversion_rate จะเก็บข้อมูลในรูปแบบ timestamp ที่ส่วนเวลาเป็นเวลา 00:00:00 ทั้งหมด

In [ ]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,19/4/2019 15:44,8.29,NaN,United Kingdom
161327,550638,22961,JAM MAKING SET PRINTED,1,19/4/2019 15:44,1.45,NaN,United Kingdom
161328,550638,22978,PANTRY ROLLING PIN,1,19/4/2019 15:44,7.46,NaN,United Kingdom
161329,550638,22982,PANTRY PASTRY BRUSH,1,19/4/2019 15:44,2.46,NaN,United Kingdom


In [ ]:
retail.describe()

,InvoiceNo,Quantity,UnitPrice,CustomerID
count,161331.000000,161331.000000,161331.000000,111388.000000
mean,543335.175688,9.936392,4.152132,15293.393471
std,4128.304528,191.110632,38.918686,1732.404778
min,536365.000000,-5368.000000,0.000000,12346.000000
25%,539750.000000,1.000000,1.250000,13851.000000
50%,543191.000000,3.000000,2.250000,15194.000000
75%,546892.000000,10.000000,4.210000,16873.000000
max,550638.000000,74215.000000,13541.330000,18283.000000


In [ ]:
# ก็อปปี้ column InvoiceDate เก็บเอาไว้ เผื่อได้ใช้ในอนาคต (อันใกล้) --> ไม่งั้นข้อมูล timestamp ของเราจะหายไป ಥ_ಥ
retail['InvoiceTimestamp'] = retail['InvoiceDate'] #newly added column
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom,1/12/2018 08:26
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom,1/12/2018 08:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,19/4/2019 15:44,8.29,NaN,United Kingdom,19/4/2019 15:44
161327,550638,22961,JAM MAKING SET PRINTED,1,19/4/2019 15:44,1.45,NaN,United Kingdom,19/4/2019 15:44
161328,550638,22978,PANTRY ROLLING PIN,1,19/4/2019 15:44,7.46,NaN,United Kingdom,19/4/2019 15:44
161329,550638,22982,PANTRY PASTRY BRUSH,1,19/4/2019 15:44,2.46,NaN,United Kingdom,19/4/2019 15:44


In [ ]:
# แปลงให้ InvoiceDate ใน retail กับ date ใน conversion_rate มีเฉพาะส่วน date ก่อน
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

retail.head()
#conversion_rate['date']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26


In [ ]:
# รวม 2 dataframe เข้าด้วยกันด้วยคำสั่ง merge
final_df = retail.merge(conversion_rate, how="left", left_on="InvoiceDate", right_on="date")
final_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
...,...,...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,2019-04-19,8.29,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161327,550638,22961,JAM MAKING SET PRINTED,1,2019-04-19,1.45,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161328,550638,22978,PANTRY ROLLING PIN,1,2019-04-19,7.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622
161329,550638,22982,PANTRY PASTRY BRUSH,1,2019-04-19,2.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622


พอ join ข้อมูลได้แล้ว เราก็ มา คูณ currency conversion กัน (UnitPrice * Rate)

In [ ]:
#เพิ่ม column 'THBPrice' ที่เกิดจาก column UnitPrice * Rate using pandas apply function 

#First Approach
final_df['THBPrice'] = final_df['UnitPrice'] * final_df['Rate']
final_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate,THBPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,111.614775
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,120.368875
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
...,...,...,...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,2019-04-19,8.29,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,342.892638
161327,550638,22961,JAM MAKING SET PRINTED,1,2019-04-19,1.45,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,59.975190
161328,550638,22978,PANTRY ROLLING PIN,1,2019-04-19,7.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,308.562012
161329,550638,22982,PANTRY PASTRY BRUSH,1,2019-04-19,2.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,101.751012


In [ ]:
def convert_rate(price, rate): # third approach
  return price * rate

final_df['THBPrice'] = final_df.apply(lambda x: convert_rate(x["UnitPrice"],x["Rate"]), axis=1) #axis=1 column

#final_df['THBPrice'] = final_df.apply(lambda x: x["UnitPrice"] * x["Rate"], axis=1) #axis=1 column # second approach
final_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate,THBPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,111.614775
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,120.368875
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705,148.381995
...,...,...,...,...,...,...,...,...,...,...,...,...
161326,550638,22960,JAM MAKING SET WITH JARS,1,2019-04-19,8.29,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,342.892638
161327,550638,22961,JAM MAKING SET PRINTED,1,2019-04-19,1.45,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,59.975190
161328,550638,22978,PANTRY ROLLING PIN,1,2019-04-19,7.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,308.562012
161329,550638,22982,PANTRY PASTRY BRUSH,1,2019-04-19,2.46,NaN,United Kingdom,19/4/2019 15:44,2019-04-19,41.3622,101.751012


## Save to CSV

In [ ]:
final_df.to_csv("output.csv", index = False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
